In [2]:
import numpy as np
import sys

In [78]:
class PartyNN(object):
    
    def __init__(self, learning_rate=0.1):
        self.weights_0_1 = np.random.normal(0.0, 2 ** -0.5, (2, 3))
        self.weights_1_2 = np.random.normal(0.0, 1, (1, 2))
        self.sigmoid_mapper = np.vectorize(self.sigmoid)  # 1/1=exp**(-x)
        self.learning_rate = np.array([learning_rate])
        
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))
    
    def predict(self, inputs):
        inputs_1 = np.dot(self.weights_0_1, inputs) # скаляр умнож  3 нейрона по два веса на входа из 3 значен
        outputs_1 = self.sigmoid_mapper(inputs_1)
        
        inputs_2 = np.dot(self.weights_1_2, outputs_1)
        outputs_2 = self.sigmoid_mapper(inputs_2)
        return outputs_2
    
    def train(self, inputs, expected_predict):     
        inputs_1 = np.dot(self.weights_0_1, inputs)
        outputs_1 = self.sigmoid_mapper(inputs_1)
        
        inputs_2 = np.dot(self.weights_1_2, outputs_1)
        outputs_2 = self.sigmoid_mapper(inputs_2)
        actual_predict = outputs_2[0]             # 'это вход значения с бинарного ответа получившегося
        
        error_layer_2 = np.array([actual_predict - expected_predict])
        gradient_layer_2 = actual_predict * (1 - actual_predict)
        weights_delta_layer_2 = error_layer_2 * gradient_layer_2  
        self.weights_1_2 -= (np.dot(weights_delta_layer_2, outputs_1.reshape(1, len(outputs_1)))) * self.learning_rate
        
        error_layer_1 = weights_delta_layer_2 * self.weights_1_2
        gradient_layer_1 = outputs_1 * (1 - outputs_1)
        weights_delta_layer_1 = error_layer_1 * gradient_layer_1
        self.weights_0_1 -= np.dot(inputs.reshape(len(inputs), 1), weights_delta_layer_1).T  * self.learning_rate

In [79]:
def MSE(y, Y):
    return np.mean((y-Y)**2)

In [82]:
train = [
    ([0, 0, 0], 0),
    ([0, 0, 1], 1),
    ([0, 1, 0], 0),
    ([0, 1, 1], 0),
    ([1, 0, 0], 1),
    ([1, 0, 1], 0),
    ([1, 1, 0], 0),
    ([1, 1, 1], 0),
]
for e in range(1):
    inputss = []
    cor = []
    for insta, cor_pred in train:
        #print(insta,'   ',cor_pred)
        #inputss.append(np.array(insta))
        inputss.append(insta)
    print(inputss)    

[[0, 0, 0], [0, 0, 1], [0, 1, 0], [0, 1, 1], [1, 0, 0], [1, 0, 1], [1, 1, 0], [1, 1, 1]]


In [81]:
#epochs = 5000
#learning_rate = 0.05
epochs = 6000
learning_rate = 0.08

network = PartyNN(learning_rate=learning_rate)

for e in range(epochs): # Счетчик от 0 до 6000
    inputs_ = []          # Под масив numpy
    correct_predictions = []
    for input_stat, correct_predict in train:
        network.train(np.array(input_stat), correct_predict) # Запуск нейронки обучения передаем выборку и ответы
        inputs_.append(np.array(input_stat)) # Добавляес масив numpy
        correct_predictions.append(np.array(correct_predict))
    # Тут считаем средне кварптич ошибку то что нейронка выдала и готовые ответы все это разность среднее(и **2)
    train_loss = MSE(network.predict(np.array(inputs_).T), np.array(correct_predictions))
    sys.stdout.write("\rProgress: {}, Training loss: {}".format(str(100 * e/float(epochs))[:4], str(train_loss)[:5]))

Progress: 99.9, Training loss: 0.001

In [84]:
for input_stat, correct_predict in train:
    print("For input: {} the prediction is: {}, expected: {}".format(
        str(input_stat), 
        str(network.predict(np.array(input_stat)) > .5),
        str(correct_predict == 1)))

For input: [0, 0, 0] the prediction is: [False], expected: False
For input: [0, 0, 1] the prediction is: [ True], expected: True
For input: [0, 1, 0] the prediction is: [False], expected: False
For input: [0, 1, 1] the prediction is: [False], expected: False
For input: [1, 0, 0] the prediction is: [ True], expected: True
For input: [1, 0, 1] the prediction is: [ True], expected: False
For input: [1, 1, 0] the prediction is: [False], expected: False
For input: [1, 1, 1] the prediction is: [False], expected: False


In [83]:
for input_stat, correct_predict in train:
    print("For input: {} the prediction is: {}, expected: {}".format(
        str(input_stat), 
        str(network.predict(np.array(input_stat))),
        str(correct_predict == 1)))

For input: [0, 0, 0] the prediction is: [0.09234344], expected: False
For input: [0, 0, 1] the prediction is: [0.94411969], expected: True
For input: [0, 1, 0] the prediction is: [0.00020336], expected: False
For input: [0, 1, 1] the prediction is: [0.00056595], expected: False
For input: [1, 0, 0] the prediction is: [0.94403003], expected: True
For input: [1, 0, 1] the prediction is: [0.98124774], expected: False
For input: [1, 1, 0] the prediction is: [0.00055662], expected: False
For input: [1, 1, 1] the prediction is: [0.02827175], expected: False


In [81]:
network.weights_0_1 # Тут смотрим на вес

array([[ 1.92542631, -3.86526729,  1.95739156],
       [-2.27707213,  5.15421872, -2.26037546]])

In [82]:
network.weights_1_2

array([[ 4.19855106, -8.7723603 ]])